# YOLOv5 Transfer Learning

## General Parameters
Fill out locations of data and yolo repo.

In [3]:
# yolov3 project location
yolo_directory = './'

# Darknet formatted data location
data_directory = '../darknet'

## Hyperparameters

In [1]:
batch_size = 16
epochs = 500
img_size = 224

## Run Training

In [4]:
import subprocess
from os.path import join

script = join(yolo_directory, 'train.py')
data = join(data_directory, 'config.yaml')
weights = join(yolo_directory, 'weights', 'yolov5s.pt')
hyper = join(yolo_directory, 'data', 'hyp.finetune.yaml')

command = 'python3 {} --batch {} --weights {} --data {} --epochs {} --cache --img {} --hyp {}'\
            .format(script, batch_size, weights, data, epochs, img_size, hyper)

print('Run this command in terminal:')
print(command)

Run this command in terminal:
python3 ./train.py --batch 16 --weights ./weights/yolov5s.pt --data ../darknet/config.yaml --epochs 500 --cache --img 224 --hyp ./data/hyp.finetune.yaml


# Load and Test Model
Test the model on some images in the validation data.

### Define Dataloader class

In [3]:
import cv2
import random
import time
import torch
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from utils.datasets import letterbox
from utils.torch_utils import select_device, load_classifier, time_synchronized
from models.experimental import attempt_load
from utils.plots import plot_one_box
from utils.general import check_img_size, non_max_suppression, apply_classifier, scale_coords, xyxy2xywh, \
    strip_optimizer, set_logging, increment_path

class DataLoader:
    def __init__(self, img_paths, img_size=512):
        self.img_paths = img_paths
        self.img_size = img_size
        self.nf = len(img_paths)
        
    def __iter__(self):
        self.count = 0
        return self

    def __next__(self):
        if self.count == self.nf:
            raise StopIteration
            
        # Read image
        path = self.img_paths[self.count]
        self.count += 1
        img0 = cv2.imread(path)  # BGR
        assert img0 is not None, 'Image Not Found ' + path
        print('image %g/%g %s: ' % (self.count, self.nf, path), end='')

        # Padded resize
        img = letterbox(img0, new_shape=self.img_size)[0]

        # Convert
        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        img = np.ascontiguousarray(img)

        return path, img, img0

    def __len__(self):
        return self.nf  # number of files
    
def detect(img_paths):

    # Get names and colors
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]
    
    # Set Dataloader
    dataset = DataLoader(img_paths)

    # Run inference
    t0 = time.time()
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
    _ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
    for path, img, im0s in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        pred = model(img, augment=False)[0]

        # Apply NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres)
        t2 = time_synchronized()

        # Process detections
        results = []
        for i, det in enumerate(pred):  # detections per image
            p, s, im0 = Path(path), '', im0s

            s += '%gx%g ' % img.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += '%g %ss, ' % (n, names[int(c)])  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    label = '%s %.2f' % (names[int(cls)], conf)
                    plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=3)
                results.append(im0)

            # Print time (inference + NMS)
            print('%sDone. (%.3fs)' % (s, t2 - t1))

        # Show results
        cv2.imshow(str(p), im0)
        if cv2.waitKey(1) == ord('q'):  # q to quit
            raise StopIteration
                    
    print('Done. (%.3fs)' % (time.time() - t0))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

### Load Model

In [5]:
from utils.torch_utils import select_device, load_classifier, time_synchronized
from models.experimental import attempt_load
from utils.general import check_img_size, non_max_suppression, apply_classifier, scale_coords, xyxy2xywh, \
    strip_optimizer, set_logging, increment_path

device = select_device('')
half = device.type != 'cpu'  # half precision only supported on CUDA

# Load model
imgsz = 512
model = attempt_load(r'../runs/train/exp2/weights/best.pt', map_location=device)
imgsz = check_img_size(imgsz, s=model.stride.max())
if half:
    model.half()

Fusing layers... 


### Run Inference
Select a number of random images from the validation set to run inference on.

In [6]:
# Inference parameters
imgsz = 512
conf_thres = 0.25
iou_thres = 0.45

In [7]:
import glob
from random import shuffle
from os.path import join

img_files = glob.glob(join(data_directory, 'images', 'val','*.jpg'))
shuffle(img_files)
img_files = img_files[:5]

results = detect(img_files)

image 1/5 ../darknet2\images\val\pedestrianCrossing_1333395974.avi_image11.jpg: 288x512 1 pedestrianCrossings, Done. (0.063s)
image 2/5 ../darknet2\images\val\addedLane_1323821747.avi_image2.jpg: 352x512 1 aheads, Done. (0.048s)
image 3/5 ../darknet2\images\val\merge_1323822076.avi_image4.jpg: 352x512 1 merges, Done. (0.031s)
image 4/5 ../darknet2\images\val\addedLane_1323816988.avi_image3.jpg: 352x512 1 addedLanes, Done. (0.039s)
image 5/5 ../darknet2\images\val\keepRight_1333393691.avi_image20.jpg: 288x512 1 keepRights, 1 speedLimits, Done. (0.031s)
Done. (3.620s)
